train/test.csv
<table width=100%>
<tr>
<th>Column name</th><th>Description</th><th>Data type</th>
</tr>
<tr>
<td>date_time</td><td>Timestamp</td><td>string</td>
</tr>
<tr>
<td>site_name</td><td>ID of the Expedia point of sale (i.e. Expedia.com, Expedia.co.uk, Expedia.co.jp, ...)</td><td>int</td>
</tr>
<tr>
<td>posa_continent</td><td>ID of continent associated with site_name</td><td>int</td>
</tr>
<tr>
<td>user_location_country</td><td>The ID of the country the customer is located</td><td>int</td>
</tr>
<tr>
<td>user_location_region</td><td>The ID of the region the customer is located</td><td>int</td>
</tr>
<tr>
<td>user_location_city</td><td>The ID of the city the customer is located</td><td>int</td>
</tr>
<tr>
<td>orig_destination_distance</td><td>Physical distance between a hotel and a customer at the time of search. A null means the distance could not be calculated</td><td>double</td>
</tr>
<tr>
<td>user_id</td><td>ID of user</td><td>int</td>
</tr>
<tr>
<td>is_mobile</td><td>1 when a user connected from a mobile device, 0 otherwise</td><td>tinyint</td>
</tr>
<tr>
<td>is_package</td><td>1 if the click/booking was generated as a part of a package (i.e. combined with a flight), 0 otherwise</td><td>int</td>
</tr>
<tr>
<td>channel</td><td>ID of a marketing channel</td><td>int</td>
</tr>
<tr>
<td>srch_ci</td><td>Checkin date</td><td>string</td>
</tr>
<tr>
<td>srch_co</td><td>Checkout date</td><td>string</td>
</tr>
<tr>
<td>srch_adults_cnt</td><td>The number of adults specified in the hotel room</td><td>int</td>
</tr>
<tr>
<td>srch_children_cnt</td><td>The number of (extra occupancy) children specified in the hotel room</td><td>int</td>
</tr>
<tr>
<td>srch_rm_cnt</td><td>The number of hotel rooms specified in the search</td><td>int</td>
</tr>
<tr>
<td>srch_destination_id</td><td>ID of the destination where the hotel search was performed</td><td>int</td>
</tr>
<tr>
<td>srch_destination_type_id</td><td>Type of destination</td><td>int</td>
</tr>
<tr>
<td>hotel_continent</td><td>Hotel continent</td><td>int</td>
</tr>
<tr>
<td>hotel_country</td><td>Hotel country</td><td>int</td>
</tr>
<tr>
<td>hotel_market</td><td>Hotel market</td><td>int</td>
</tr>
<tr>
<td>is_booking</td><td>1 if a booking, 0 if a click</td><td>tinyint</td>
</tr>
<tr>
<td>cnt</td><td>Numer of similar events in the context of the same user session</td><td>bigint</td>
</tr>
<tr>
<td>hotel_cluster</td><td>ID of a hotel cluster</td><td>int</td>
</tr>
</table>

destinations.csv
<table width=100%>
<tr>
<th>Column name</th><th>Description</th><th>Data type</th>
</tr>
<tr>
<td>srch_destination_id</td><td>ID of the destination where the hotel search was performed</td><td>int</td>
</tr>
<tr>
<td>d1-d149</td><td>latent description of search regions</td><td>double</td>
</tr>
</table>

**The prediction required 5 clusters as the pred result, and the earlier the correct cluster appears in the list of 5, the high the result will score**

In [28]:
import pandas as pd
import numpy as np
from IPython.display import display
import ml_metrics as metrics

%matplotlib inline

In [4]:
destinations = pd.read_csv('data/destinations_expedia.csv')
# Get a smaller train and test file first to test the algorithms before using the entire dataset
train = pd.read_csv('data/train_small.csv')
test = pd.read_csv('data/test_small.csv')

In [5]:
train.shape

(49999, 24)

In [6]:
test.shape

(9999, 22)

In [10]:
pd.options.display.max_columns = None
display(train[:5])

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,9,2014-08-27,2014-08-31,2,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,9,2014-08-29,2014-09-02,2,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,9,2014-08-29,2014-09-02,2,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,3,2014-11-23,2014-11-28,2,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,3,2014-11-23,2014-11-28,2,0,1,14984,1,0,1,2,50,1457,21


In [11]:
display(test[:5])

,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
0,0,2015-09-03 17:09:54,2,3,66,174,37449,5539.0567,1,1,0,3,2016-05-19,2016-05-23,2,0,1,12243,6,6,204,27
1,1,2015-09-24 17:38:35,2,3,66,174,37449,5873.2923,1,1,0,10,2016-05-12,2016-05-15,2,0,1,14474,7,6,204,1540
2,2,2015-06-07 15:53:02,2,3,66,142,17440,3975.9776,20,0,0,1,2015-07-26,2015-07-27,4,0,1,11353,1,2,50,699
3,3,2015-09-14 14:49:10,2,3,66,258,34156,1508.5975,28,0,1,10,2015-09-14,2015-09-16,2,0,1,8250,1,2,50,628
4,4,2015-07-17 09:32:04,2,3,66,467,36345,66.7913,50,0,0,0,2015-07-22,2015-07-23,2,0,1,11812,1,2,50,538


In [13]:
# See how many searches are there in each hotel cluster
train.hotel_cluster.value_counts()

91    1387
41    1127
48    1027
64     986
65     856
42     798
70     784
5      769
25     768
46     728
97     715
83     705
59     694
21     693
37     685
98     680
95     679
58     652
2      649
18     648
30     645
82     642
50     639
16     637
72     628
28     621
68     615
62     612
9      607
10     596
      ... 
38     373
12     365
7      365
86     363
19     358
43     358
89     352
81     348
23     344
31     342
79     338
14     334
63     334
32     326
66     325
3      316
45     310
80     303
49     301
87     287
71     274
93     274
60     258
35     224
75     224
24     210
53     185
27     163
88     126
74      60
Name: hotel_cluster, dtype: int64

In [16]:
# In order to see if the user_ids in test.csv is a subset of the ones in train.csv, we'll need to entire datasets
train = pd.read_csv('data/train_expedia.csv')
test = pd.read_csv('data/test_expedia.csv')

In [17]:
test_ids = set(test.user_id.unique())
train_ids = set(train.user_id.unique())
intersection_count = len(test_ids & train_ids)
intersection_count == len(test_ids)

True

**Looking into destinations.csv file**

In [ ]:
destinations[:5]

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=3)
dest_small = pd.DataFrame(pca.fit_transform(destinations.drop('srch_destination_id',axis=1)))
dest_small['srch_destination_id'] = destinations['srch_destination_id']

In [ ]:
destinations[:5]

**True shows that all user ids in test.csv are indeed a subset of those of train.csv**

In [19]:
# Using date_time column to generate new features
#train.loc[0].date_time.split(' ')[0].split('-')[1]
#train['month'] = train.date_time.split(' ')[0].split('-')[1]
train['year'] = pd.to_datetime(train['date_time']).dt.year
train['month'] = pd.to_datetime(train['date_time']).dt.month

In [43]:
# Convert non-numerical fields into numerical
train['date_time'] = pd.to_datetime(train['date_time'])
train['srch_ci'] = pd.to_datetime(train['srch_ci'], format='%Y-%m-%d', errors='coerce')
train['srch_co'] = pd.to_datetime(train['srch_co'], format='%Y-%m-%d', errors='coerce')

In [53]:
props = {}
for prop in ["month", "day", "hour", "minute", "dayofweek", "quarter"]:
    props[prop] = getattr(train["date_time"].dt, prop)

In [54]:
carryover = [p for p in train.columns if p not in ["date_time", "srch_ci", "srch_co"]]
for prop in carryover:
    props[prop] = train[prop]

In [56]:
# Calculate the days between check-in and check-out dates
date_props = ["month", "day", "dayofweek", "quarter"]
for prop in date_props:
    props["ci_{0}".format(prop)] = getattr(train["srch_ci"].dt, prop)
    props["co_{0}".format(prop)] = getattr(train["srch_co"].dt, prop)
props["stay_span"] = (train["srch_co"] - train["srch_ci"]).astype('timedelta64[h]')

In [60]:
train_new = pd.DataFrame(props).join(dest_small, on="srch_destination_id", how='left', rsuffix="dest")

In [61]:
train_new.shape

(37670293, 41)

In [62]:
# Get a smaller dataset using random sampling
sel_train = train_new.sample(n=50000, random_state=123)

In [63]:
# sel_train.shape

In [64]:
# Split the sample set into training and testing sets, t1 is for training and t2 is for testing
t1 = sel_train[((sel_train.year == 2013) | ((sel_train.year == 2014) & (sel_train.month < 8)))]
t2 = sel_train[((sel_train.year == 2014) & (sel_train.month >= 8))]

In [65]:
# Since the real testing data contains only booking events, we'll remove the clicking events from t2
t2 = t2[t2.is_booking==1]